In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
from collections import defaultdict
import pickle

In [2]:
movies = pd.read_csv('data/IMDB_movie_details.csv').reset_index(drop=True)
reviews = pd.read_json('data/IMDB_reviews.json', lines = True)

movies.movie_id = movies.movie_id.astype('str')
reviews.movie_id = reviews.movie_id.astype('str')

In [3]:
from bs4 import BeautifulSoup
import requests
import random

user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0',
    'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36 Edg/99.0.1150.46',
    'Mozilla/5.0 (Linux; Android 10; SM-G981B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.58 Mobile Safari/537.36']

# add a bar to show the progress of the scraping
count = 0

#scrape the movie title
def get_title(movie_id):
    response = requests.get(f"https://www.imdb.com/title/{movie_id}/", headers = {'User-Agent': random.choice(user_agents_list)})
    soup =  BeautifulSoup(response.content, 'html.parser')
    global count 
    count += 1
    if count % 100 == 0:
        print(count)
    try:
        title = soup.find('h1').text
        return title
    except:
        return None

movies.loc[movies.title.isna(), 'title'] = movies.loc[movies.title.isna(), 'movie_id'].apply(get_title)

In [4]:
df = pd.merge(movies, reviews, on='movie_id')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573906 entries, 0 to 573905
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   movie_id        573906 non-null  object 
 1   plot_summary    573906 non-null  object 
 2   duration        573906 non-null  object 
 3   genre           573906 non-null  object 
 4   rating_x        573906 non-null  float64
 5   release_date    573906 non-null  object 
 6   plot_synopsis   538828 non-null  object 
 7   title           573906 non-null  object 
 8   review_date     573906 non-null  object 
 9   user_id         573906 non-null  object 
 10  is_spoiler      573906 non-null  bool   
 11  review_text     573906 non-null  object 
 12  rating_y        573906 non-null  int64  
 13  review_summary  573906 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(11)
memory usage: 57.5+ MB


In [5]:
df = pd.merge(movies, reviews, on='movie_id')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573906 entries, 0 to 573905
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   movie_id        573906 non-null  object 
 1   plot_summary    573906 non-null  object 
 2   duration        573906 non-null  object 
 3   genre           573906 non-null  object 
 4   rating_x        573906 non-null  float64
 5   release_date    573906 non-null  object 
 6   plot_synopsis   538828 non-null  object 
 7   title           573906 non-null  object 
 8   review_date     573906 non-null  object 
 9   user_id         573906 non-null  object 
 10  is_spoiler      573906 non-null  bool   
 11  review_text     573906 non-null  object 
 12  rating_y        573906 non-null  int64  
 13  review_summary  573906 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(11)
memory usage: 57.5+ MB


## Understand all present characters

### Lower Everything

In [6]:
df['plot_summary'] = df['plot_summary'].astype(str)
df['plot_synopsis'] = df['plot_synopsis'].astype(str)
df['review_text'] = df['review_text'].astype(str)
df['review_summary  '] = df['plot_summary'].astype(str)

df['plot_summary'] = df['plot_summary'].apply(lambda x: x.strip().lower())
df['plot_synopsis'] = df['plot_synopsis'].apply(lambda x: x.strip().lower())
df['review_text'] = df['review_text'].apply(lambda x: x.strip().lower())
df['review_summary  '] = df['plot_summary'].apply(lambda x: x.strip().lower())

### Create dictionaries with char counts

#### Plot Summary

In [7]:
plot_summary_corpus = ' '.join(df['plot_summary'].values)

In [8]:
plot_summary_char = defaultdict(int)
for char in plot_summary_corpus:
    plot_summary_char[char] += 1
plot_summary_char

defaultdict(int,
            {'f': 6087680,
             'o': 19939472,
             'r': 18363141,
             'm': 7155040,
             'e': 34057353,
             ' ': 67761855,
             'c': 7985431,
             'i': 20820824,
             'a': 23391550,
             'n': 20694678,
             'l': 11881587,
             'y': 5517676,
             's': 18828467,
             't': 24657037,
             ',': 3438950,
             'j': 847028,
             'k': 2447851,
             'g': 6056235,
             'd': 10107315,
             'w': 5846571,
             'h': 15669430,
             'v': 3120661,
             'u': 6828339,
             'x': 484477,
             'p': 4888714,
             'b': 5028768,
             '.': 3131464,
             "'": 817460,
             '\n': 546548,
             '0': 190909,
             '4': 71824,
             '1': 205057,
             '@': 130049,
             '(': 229152,
             ')': 227986,
             '-': 717037,
          

In [9]:
len(plot_summary_char)

80

Strange chars: '@': 130051, 'é': 18733, '{': 8218, '}': 8218, (probably symmetric), 'ö': 2415 (in characters names?), '_': 22686, '#': 982, '+': 521, '%': 2479, '½': 553, other accented letters, '~': 313, but NO EMOJIS

In [23]:
with open('plot_summary_char.csv', 'w', encoding='utf-8') as f:
    for char in plot_summary_char:
        f.write("'" + str(char) + "'" + ',' + str(plot_summary_char[char]) + '\n')

#### Plot Synopsis

In [9]:
plot_synopsis_corpus = ' '.join(df['plot_synopsis'].values)

In [10]:
plot_synopsis_char = defaultdict(int)
for char in plot_synopsis_corpus:
    plot_synopsis_char[char] += 1
plot_synopsis_char

defaultdict(int,
            {'j': 13188516,
             'a': 401718343,
             'c': 128067287,
             'k': 53366735,
             ' ': 1039692092,
             'r': 288793769,
             'y': 78764761,
             'n': 332238602,
             '(': 6726947,
             'f': 93035796,
             'o': 329958807,
             'd': 175669012,
             ')': 6730972,
             'i': 326389502,
             's': 341922817,
             '"': 7955875,
             'w': 84554238,
             'g': 106315851,
             'v': 46555408,
             't': 428495565,
             'l': 197808316,
             'h': 303231103,
             'm': 116593715,
             '.': 58386879,
             'e': 580961408,
             'p': 84645090,
             'u': 113479125,
             ',': 57773741,
             "'": 19187203,
             'b': 79920251,
             'z': 6421824,
             ':': 1071986,
             'q': 3296493,
             'x': 8326823,
             '-': 676

In [26]:
len(plot_synopsis_char)

106

In [24]:
with open('plot_synopsis_char.csv', 'w', encoding='utf-8') as f:
    for char in plot_synopsis_char:
        f.write("'" + str(char) + "'" + ',' + str(plot_synopsis_char[char]) + '\n')

#### Review Text

In [29]:
review_text_corpus = ' '.join(df['review_text'].values)

In [30]:
review_text_char = defaultdict(int)
for char in review_text_corpus:
    review_text_char[char] += 1
review_text_char

defaultdict(int,
            {'t': 63195634,
             'h': 36097105,
             'e': 80245116,
             ' ': 148872335,
             's': 44577209,
             'c': 18833956,
             'o': 49411555,
             'n': 44054574,
             'd': 22957867,
             'm': 18667325,
             'l': 29396684,
             'a': 53515839,
             'y': 13565539,
             'v': 8751976,
             'i': 51890690,
             'f': 14811115,
             '(': 788747,
             "'": 3325957,
             'u': 17263124,
             'r': 36884463,
             'b': 10446449,
             'g': 13949620,
             ')': 814894,
             ',': 7081270,
             'p': 11231795,
             'w': 13227883,
             '.': 8510985,
             'k': 5686869,
             'z': 574136,
             'j': 1637966,
             'x': 1193204,
             '-': 1414081,
             '?': 386803,
             ';': 157778,
             '/': 200945,
             '"': 1318

In [31]:
len(review_text_char)

705

In [32]:
with open('review_text.csv', 'w', encoding='utf-8') as f:
    for char in review_text_char:
        f.write("'" + str(char) + "'" + ',' + str(review_text_char[char]) + '\n')

Full of emojis and non alphanumerical characters!!!

#### Review Summary

In [33]:
review_summary_corpus = ' '.join(df['review_summary'].values)

In [34]:
review_summary_char = defaultdict(int)
for char in review_summary_corpus:
    review_summary_char[char] += 1
review_summary_char

defaultdict(int,
            {'d': 482060,
             'e': 1839782,
             'c': 389978,
             'n': 989397,
             't': 1325593,
             ' ': 3436536,
             'C': 56065,
             'l': 716823,
             'a': 1115209,
             'y': 321739,
             'h': 577906,
             'r': 896603,
             'i': 1187338,
             'J': 16093,
             'k': 120181,
             'R': 43370,
             'p': 245646,
             'I': 93538,
             "'": 78008,
             's': 894939,
             'f': 335702,
             'o': 1199434,
             'b': 231659,
             'm': 445694,
             'w': 211901,
             'x': 38806,
             'O': 53912,
             'u': 428202,
             '!': 127665,
             'P': 43847,
             'G': 58385,
             '.': 275640,
             'H': 45186,
             'F': 57983,
             'A': 142482,
             'T': 124090,
             'D': 55704,
             'g': 327399,
 

In [35]:
len(review_summary_char)

265

In [36]:
with open('review_summary.csv', 'w', encoding='utf-8') as f:
    for char in review_summary_char:
        f.write("'" + str(char) + "'" + ',' + str(review_summary_char[char]) + '\n')

Again full of strange characters!!!!